In [13]:
from numba import cuda
import numpy as np
import math

#Compute ∑∑p(x,y)/(p(x)*p(y)) over all possible x and y

#mxrec is the reciprocal/inverse and transpose of the marginal distribution of x
#jxy is the joint distribution of x and y
#myrec is the reciprocal/inverse of marginal distribution of y
# ∑∑p(x,y)/(p(x)*p(y)) is stored in mxrec_X_jxy_X_myrec
# which has the form (Nx, Ny)

#The following function calculates the marginal using a single GPU
@cuda.jit
def computeSumMIGPU( mxrec, jxy, myrec, mxrec_X_jxy_X_myrec ):
    
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    
    if ( row < mxrec_X_jxy_X_myrec.shape[0] and col < mxrec_X_jxy_X_myrec.shape[1] ):
        val  =  mxrec[0,row*2  ]   * jxy[row*2  ,col*2  ]    * myrec[col*2  ,0]
        val +=  mxrec[0,row*2+1]   * jxy[row*2+1,col*2  ]    * myrec[col*2  ,0]  
        val +=  mxrec[0,row*2  ]   * jxy[row*2  ,col*2+1]    * myrec[col*2+1,0]
        val +=  mxrec[0,row*2+1]   * jxy[row*2+1,col*2+1]    * myrec[col*2+1,0] 
        mxrec_X_jxy_X_myrec[row,col] = val

#Compute ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) over all possible x y and z

#jxzrec is the reciprocal/inverse of the joint distrubtion of xz
#jxyz is the joint distribution of x, y, and z
#mz is the marginal distribution of z
#jyzrec is the reciprocal/inverse of the joint distribution of yz
# ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) is stored in jxzrec_X_jxyz_X_jyzrec_X_mz
# which has the form (Nx, Ny, Nz)

#The following function calculates the marginal using a single GPU
@cuda.jit
def computeSumCMIGPU( jxzrec, jxyz, jyzrec, mz, jxzrec_X_jxyz_X_jyzrec_X_mz ):
    
    row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    z   = cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z
    
    if (    row < jxzrec_X_jxyz_X_jyzrec_X_mz.shape[0] 
        and col < jxzrec_X_jxyz_X_jyzrec_X_mz.shape[1] 
        and z   < jxzrec_X_jxyz_X_jyzrec_X_mz.shape[2] ):
        
        val   = jxzrec[row*2,   2*z  ]   * jxyz[row*4,  col*2  , z]   * jyzrec[col*2,   2*z  ]  * mz[2*z  , 0]
        val  += jxzrec[row*2,   2*z+1]   * jxyz[row*4+1,col*2  , z]   * jyzrec[col*2,   2*z+1]  * mz[2*z+1, 0]
        val  += jxzrec[row*2+1, 2*z  ]   * jxyz[row*4+2,col*2  , z]   * jyzrec[col*2,   2*z  ]  * mz[2*z  , 0]
        val  += jxzrec[row*2+1, 2*z+1]   * jxyz[row*4+3,col*2  , z]   * jyzrec[col*2,   2*z+1]  * mz[2*z+1, 0]
        val  += jxzrec[row*2,   2*z  ]   * jxyz[row*4,  col*2+1, z]   * jyzrec[col*2+1, 2*z  ]  * mz[2*z,   0]
        val  += jxzrec[row*2,   2*z+1]   * jxyz[row*4+1,col*2+1, z]   * jyzrec[col*2+1, 2*z+1]  * mz[2*z+1, 0]
        val  += jxzrec[row*2+1, 2*z  ]   * jxyz[row*4+2,col*2+1, z]   * jyzrec[col*2+1, 2*z  ]  * mz[2*z,   0]
        val  += jxzrec[row*2+1, 2*z+1]   * jxyz[row*4+3,col*2+1, z]   * jyzrec[col*2+1, 2*z+1]  * mz[2*z+1, 0]
        jxzrec_X_jxyz_X_jyzrec_X_mz[row,col,z] = val

#Calculation of the joint distribution of a sets of binary variables, X and Y

#X is the mxn matrix of observations x classes, where Xij, codes the presence 
#of a stimulus of class j at observation i

#Y is the mxn matrix of observations x voxels, where Yij, codes the presence 
#of some binary feature for voxel j at observation i
#Y contains all of the voxels which may contain information (i.e. with some mask of interest)

#The output is stored in the matrix j of the form
#( 2*Nclasses, 2*Nvoxels ) or ( 2*x.shape[1], 2*y.shape[1] )

#P( X = 0, Y = 0 ) will be stored at 2*Nx,   2*Ny
#P( X = 0, Y = 1 ) will be stored at 2*Nx,   2*Ny+1
#P( X = 1, Y = 0 ) will be stored at 2*Nx+1, 2*Ny
#P( X = 1, Y = 1 ) will be stored at 2*Nx+1, 2*Ny+1

#If the inverse is true, then reciprocal values of P(X=i,Y=j) 
#for i,j element of {0,1} are returned

#If the tranpose is true, then the vector j must have shape ( 2*Nvoxels, 2*Nclasses ) 

#The following function calculates the marginal using a single GPU
@cuda.jit    
def jointGPU( x, y, j, inv, transpose ):
    
    colx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    coly = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    N = (1/float(x.shape[0]))
    
    if ( colx < x.shape[1] and coly < y.shape[1] ):

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 0:
                val += N
        if inv:
            if transpose:
                j[2*coly,2*colx] = 1/val
            else:
                j[2*colx,2*coly] = 1/val
        else:
            if transpose:
                j[2*coly,2*colx] = val
            else:
                j[2*colx,2*coly] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 1:
                val += N
        if inv:
            if transpose:
                j[2*coly+1,2*colx] = 1/val
            else:
                j[2*colx,2*coly+1] = 1/val
        else:
            if transpose:
                j[2*coly+1,2*colx] = val
            else:
                j[2*colx,2*coly+1] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 0:
                val += N
        if inv:
            if transpose:
                j[2*coly,2*colx+1] = 1/val
            else:
                j[2*colx+1,2*coly] = 1/val
        else:
            if transpose:
                j[2*coly,2*colx+1] = val
            else:
                j[2*colx+1,2*coly] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 1:
                val += N
        if inv:
            if transpose:
                j[2*coly+1,2*colx+1] = 1/val
            else:
                j[2*colx+1,2*coly+1] = 1/val
        else:
            if transpose:
                j[2*coly+1,2*colx+1] = val
            else:
                j[2*colx+1,2*coly+1] = val

#Calculation of the joint distribution of a sets of binary variables, X, Y, and Z

#X is the mxn matrix of observations x classes, where Xij, codes the presence 
#of a stimulus of class j at observation i

#Y is the mxn matrix of observations x voxels, where Yij, codes the presence 
#of some binary feature for voxel j at observation i
#Y contains all of the voxels which may contain information (i.e. with some mask of interest)

#Z is the mxn matrix ov observations x voxels, where Zij coes the presence
#of some binary feature for voxel j at observation i
#Z contains all of the voxels which have already been selected as features

#The output is stored in the matrix j of the form
#( 4*Nclasses, 2*Nvoxels, NSelectedVoxels ) or ( 4*x.shape[1], 2*y.shape[1], z.shape[1] )

#P( X = 0, Y = 0, Z = 0 ) will be stored at 4*Nx,   2*Ny,   Nz
#P( X = 0, Y = 0, Z = 1 ) will be stored at 4*Nx+1, 2*Ny,   Nz
#P( X = 1, Y = 0, Z = 0 ) will be stored at 4*Nx+2, 2*Ny,   Nz
#P( X = 1, Y = 0, Z = 1 ) will be stored at 4*Nx+3, 2*Ny,   Nz
#P( X = 0, Y = 1, Z = 0 ) will be stored at 4*Nx,   2*Ny+1, Nz
#P( X = 0, Y = 1, Z = 1 ) will be stored at 4*Nx+1, 2*Ny+1, Nz
#P( X = 1, Y = 1, Z = 0 ) will be stored at 4*Nx+2, 2*Ny+1, Nz
#P( X = 1, Y = 1, Z = 1 ) will be stored at 4*Nx+3, 2*Ny+1, Nz

#The following function calculates the marginal using a single GPU
@cuda.jit    
def joint3GPU( x, y, z, j ):
    
    colx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    coly = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    colz = cuda.blockIdx.z * cuda.blockDim.z + cuda.threadIdx.z
    N = (1/float(x.shape[0]))
    
    if ( colx < x.shape[1] and coly < y.shape[1] and colz < z.shape[1] ):
        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 0 and z[row,colz] == 0:
                val += N
            j[ 4*colx, 2*coly, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 0 and z[row,colz] == 1:
                val += N
            j[ 4*colx+1, 2*coly, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 1 and z[row,colz] == 0:
                val += N
            j[ 4*colx, 2*coly+1, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 0 and y[row,coly] == 1 and z[row,colz] == 1:
                val += N
            j[ 4*colx+1, 2*coly+1, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 0 and z[row,colz] == 0:
                val += N
            j[ 4*colx+2, 2*coly, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 0 and z[row,colz] == 1:
                val += N
            j[ 4*colx+3, 2*coly, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 1 and z[row,colz] == 0:
                val += N
            j[ 4*colx+2, 2*coly+1, colz ] = val

        val = 0
        for row in range(x.shape[0]):
            if x[row,colx] == 1 and y[row,coly] == 1 and z[row,colz] == 1:
                val += N
            j[ 4*colx+3, 2*coly+1, colz ] = val

#Calculation of the marginal distribution of a set of binary variables 
#X across a set observations (the rows) stored in m. 

#If the inverse is true, then reciprocal values of P(X=i) 
#for i element of {0,1} are returned

#If the tranpose is true, then the vector m must have shape (1, NumVar*2)
#Otherwise the vector m must have the shape (NumVar*2, 1)

#The probability that Var N = 0 will be stored at Nx*2
#The probability that Var N = 1 will be stored at Nx*2 + 1

#The following function calculates the marginal using a single GPU
@cuda.jit
def marginalGPU( x, m, inv, transpose ):
    
    colx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    N = float(x.shape[0])
    if ( colx < x.shape[1] ):
        val = 0
        for row in range(x.shape[0]):
            val += x[row,colx]
        if inv:
            if transpose:
                m[0,2*colx] = N/(N-val)
                m[0,2*colx+1] = N/val
            else:
                m[2*colx,0] = N/(N-val)
                m[2*colx+1,0] = N/val
        else:
            if transpose:
                m[0,2*colx] = (N-val)/N
                m[0,2*colx+1] = val/N
            else:
                m[2*colx,0] = (N-val)/N
                m[2*colx+1,0] = val/N
                

#Calculation of MI of two sets binary variables X and Y on a sinlge cpu
#Xij is the presence or absence of stimulus j for instance i
#Yij is the presence or absence of a partiuclar binary feature at voxel j for instance i
#This function uses a single GPU
def MIGPU( x, y ):
    
    import numpy as np
    
    #Define empty variables and number of instances
    Nx                       = x.shape[1]
    Ny                       = y.shape[1]
    mxrec                    = np.zeros((1   , 2*Nx),dtype=np.float32)
    myrec                    = np.zeros((2*Ny, 1   ),dtype=np.float32)
    jxy                      = np.zeros((2*Nx, 2*Ny),dtype=np.float32)
    mxrec_X_jxy_X_myrec      = np.zeros((Nx  , Ny  ),dtype=np.float32)
    
    #Copy data to GPU
    d_x                      = cuda.to_device(x)
    d_y                      = cuda.to_device(y)
    d_mxrec                  = cuda.to_device(mxrec)
    d_myrec                  = cuda.to_device(myrec)
    d_jxy                    = cuda.to_device(jxy)
    d_mxrec_X_jxy_X_myrec    = cuda.to_device(mxrec_X_jxy_X_myrec)
    
    #Define number of blocks and threads, and compute reciprocal of the marginals 
    threads_per_block = 1000 #block threads
    number_of_blocks = int( math.ceil( float(Nx) / threads_per_block ) )
    marginalGPU [ number_of_blocks, threads_per_block ] ( d_x, d_mxrec, 1, 1 )

    number_of_blocks = int( math.ceil( float(Ny) / threads_per_block ) )
    marginalGPU [ number_of_blocks, threads_per_block ] ( d_y, d_myrec, 1, 0 )

    #Compute the joint distribution
    threads_per_block = (2,500) # blocks of 2x500 threads
    number_of_blocks =  ( int(math.ceil( float(Nx) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Ny) / threads_per_block[1] ) ) )
    jointGPU [ number_of_blocks, threads_per_block ] ( d_x, d_y, d_jxy, 0, 0 )

    #Compute ∑∑p(x,y)/(p(x)*p(y))
    threads_per_block = (2,500) # blocks of 2x500 threads
    number_of_blocks =  ( int(math.ceil( float(Nx) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Ny) / threads_per_block[1] ) ) )
    computeSumMIGPU [ number_of_blocks, threads_per_block ] ( d_mxrec, d_jxy, d_myrec, d_mxrec_X_jxy_X_myrec)
    
    #Copy non log trasnformed MI data back to CPU
    d_mxrec_X_jxy_X_myrec.copy_to_host( mxrec_X_jxy_X_myrec )
    
    #Perform log2 transform on cpu and return MI
    return np.log2( mxrec_X_jxy_X_myrec )
   
#Calculation of CMI on a single GPU
def CMIGPU( x, y, z ):
    
    import numpy as np
    
    #Define empty variables based on input shapes
    Nx                              = x.shape[1]
    Ny                              = y.shape[1]
    Nz                              = z.shape[1]

    mz                              = np.zeros((2*Nz, 1       ),dtype=np.float32)
    jxzrec                          = np.zeros((2*Nx, 2*Nz    ),dtype=np.float32)
    jyzrec                          = np.zeros((2*Ny, 2*Nz    ),dtype=np.float32)
    jxyz                            = np.zeros((4*Nx, 2*Ny, Nz),dtype=np.float32)
    jxzrec_X_jxyz_X_mz_X_jyzrec     = np.zeros((Nx  , Ny  , Nz),dtype=np.float32)

    #Copy data to GPU
    d_x                             = cuda.to_device(x)
    d_y                             = cuda.to_device(y)
    d_z                             = cuda.to_device(z)
    d_mz                            = cuda.to_device(mz)
    d_jxzrec                        = cuda.to_device(jxzrec)
    d_jyzrec                        = cuda.to_device(jyzrec)
    d_jxyz                          = cuda.to_device(jxyz)
    d_jxzrec_X_jxyz_X_mz_X_jyzrec   = cuda.to_device(jxzrec_X_jxyz_X_mz_X_jyzrec)

    #Compute the marginal distribution of z
    threads_per_block = 1000 # blocks of 1000 threads
    number_of_blocks = int( math.ceil( float(Nz) / threads_per_block ) )
    marginalGPU [ number_of_blocks, threads_per_block ] ( d_z, d_mz, 0, 0 )

    #Compute the inverse joint distributions
    threads_per_block = ( 1000, 1 ) # blocks of 1000x1 threads
    number_of_blocks =  ( int(math.ceil( float(Nx) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Nz) / threads_per_block[1] ) ) )
    jointGPU [ number_of_blocks, threads_per_block ] ( d_x, d_z, d_jxzrec, 1, 0 )
    number_of_blocks =  ( int(math.ceil( float(Ny) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Nz) / threads_per_block[1] ) ) )
    jointGPU [ number_of_blocks, threads_per_block ] ( d_y, d_z, d_jyzrec, 1, 0 )

    #Compute the triple joint distribution
    threads_per_block = ( 2, 250, 2 ) # blocks of 2x250x2 threads
    number_of_blocks =  ( int(math.ceil( float(Nx) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Ny) / threads_per_block[1] ) ),
                          int(math.ceil( float(Nz) / threads_per_block[2] ) ) )
    joint3GPU [ number_of_blocks, threads_per_block ] ( d_x, d_y, d_z, d_jxyz )

    #Compute ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) over all possible x y and z
    threads_per_block = ( 2, 250, 2 ) # blocks of 2x250x2 threads
    number_of_blocks  = ( int(math.ceil( float(Nx) / threads_per_block[0] ) ), 
                          int(math.ceil( float(Ny) / threads_per_block[1] ) ),
                          int(math.ceil( float(Nz) / threads_per_block[2] ) ) )
    computeSumCMIGPU [ number_of_blocks, threads_per_block ] ( d_jxzrec, d_jxyz, d_jyzrec, d_mz, d_jxzrec_X_jxyz_X_mz_X_jyzrec )

    #Copy non log trasnformed CMI data back to CPU
    d_jxzrec_X_jxyz_X_mz_X_jyzrec.copy_to_host( jxzrec_X_jxyz_X_mz_X_jyzrec )

    #Perform log2 transform on cpu and return CMI
    return np.log2( jxzrec_X_jxyz_X_mz_X_jyzrec )


#--------------------CPU Functions---------------------------------


#Compute ∑∑p(x,y)/(p(x)*p(y)) over all possible x and y

#mxrec is the reciprocal/inverse and transpose of the marginal distribution of x
#jxy is the joint distribution of x and y
#myrec is the reciprocal/inverse of marginal distribution of y
# ∑∑p(x,y)/(p(x)*p(y)) is stored in mxrec_X_jxy_X_myrec
# which has the form (Nx, Ny)

#The following function calculates the marginal using a single CPU
def computeSumMICPU( mxrec, jxy, myrec, mxrec_X_jxy_X_myrec ):
    
    for row in range(mxrec_X_jxy_X_myrec.shape[0]):
        for col in range(mxrec_X_jxy_X_myrec.shape[1]):
            
            val  =  mxrec[0,row*2  ]   * jxy[row*2  ,col*2  ]    * myrec[col*2  ,0]
            val +=  mxrec[0,row*2+1]   * jxy[row*2+1,col*2  ]    * myrec[col*2  ,0]  
            val +=  mxrec[0,row*2  ]   * jxy[row*2  ,col*2+1]    * myrec[col*2+1,0]
            val +=  mxrec[0,row*2+1]   * jxy[row*2+1,col*2+1]    * myrec[col*2+1,0] 
            mxrec_X_jxy_X_myrec[row,col] = val


#Compute ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) over all possible x y and z

#jxzrec is the reciprocal/inverse of the joint distrubtion of xz
#jxyz is the joint distribution of x, y, and z
#mz is the marginal distribution of z
#jyzrec is the reciprocal/inverse of the joint distribution of yz
# ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) is stored in jxzrec_X_jxyz_X_jyzrec_X_mz
# which has the form (Nx, Ny, Nz)

#The following function calculates the marginal using a single CPU
def computeSumCMICPU( jxzrec, jxyz, jyzrec, mz, jxzrec_X_jxyz_X_jyzrec_X_mz ):
    
    for row in range(jxzrec_X_jxyz_X_jyzrec_X_mz.shape[0]):
        for col in range(jxzrec_X_jxyz_X_jyzrec_X_mz.shape[1]):
            for z in range(jxzrec_X_jxyz_X_jyzrec_X_mz.shape[2]):
                val   = jxzrec[row*2,   2*z  ]   * jxyz[row*4,  col*2  , z]   * jyzrec[col*2,   2*z  ]  * mz[2*z  , 0]
                val  += jxzrec[row*2,   2*z+1]   * jxyz[row*4+1,col*2  , z]   * jyzrec[col*2,   2*z+1]  * mz[2*z+1, 0]
                val  += jxzrec[row*2+1, 2*z  ]   * jxyz[row*4+2,col*2  , z]   * jyzrec[col*2,   2*z  ]  * mz[2*z  , 0]
                val  += jxzrec[row*2+1, 2*z+1]   * jxyz[row*4+3,col*2  , z]   * jyzrec[col*2,   2*z+1]  * mz[2*z+1, 0]
                val  += jxzrec[row*2,   2*z  ]   * jxyz[row*4,  col*2+1, z]   * jyzrec[col*2+1, 2*z  ]  * mz[2*z,   0]
                val  += jxzrec[row*2,   2*z+1]   * jxyz[row*4+1,col*2+1, z]   * jyzrec[col*2+1, 2*z+1]  * mz[2*z+1, 0]
                val  += jxzrec[row*2+1, 2*z  ]   * jxyz[row*4+2,col*2+1, z]   * jyzrec[col*2+1, 2*z  ]  * mz[2*z,   0]
                val  += jxzrec[row*2+1, 2*z+1]   * jxyz[row*4+3,col*2+1, z]   * jyzrec[col*2+1, 2*z+1]  * mz[2*z+1, 0]
                jxzrec_X_jxyz_X_jyzrec_X_mz[row,col,z] = val

#Calculation of the joint distribution of a sets of binary variables, X and Y

#X is the mxn matrix of observations x classes, where Xij, codes the presence 
#of a stimulus of class j at observation i

#Y is the mxn matrix of observations x voxels, where Yij, codes the presence 
#of some binary feature for voxel j at observation i
#Y contains all of the voxels which may contain information (i.e. with some mask of interest)

#The output is stored in the matrix j of the form
#( 2*Nclasses, 2*Nvoxels ) or ( 2*x.shape[1], 2*y.shape[1] )

#P( X = 0, Y = 0 ) will be stored at 2*Nx,   2*Ny
#P( X = 0, Y = 1 ) will be stored at 2*Nx,   2*Ny+1
#P( X = 1, Y = 0 ) will be stored at 2*Nx+1, 2*Ny
#P( X = 1, Y = 1 ) will be stored at 2*Nx+1, 2*Ny+1

#If the inverse is true, then reciprocal values of P(X=i,Y=j) 
#for i,j element of {0,1} are returned

#If the tranpose is true, then the vector j must have shape ( 2*Nvoxels, 2*Nclasses ) 

#The following function calculates the marginal using a single CPU
def jointCPU( x, y, j, inv, transpose ):
    
    N = (1/float(x.shape[0]))
    for colx in range(x.shape[1]):
        for coly in range(y.shape[1]):
            val = 0
            for row in range(x.shape[0]):
                if x[row,colx] == 0 and y[row,coly] == 0:
                    val += N
            if inv:
                if transpose:
                    j[2*coly,2*colx] = 1/val
                else:
                    j[2*colx,2*coly] = 1/val
            else:
                if transpose:
                    j[2*coly,2*colx] = val
                else:
                    j[2*colx,2*coly] = val

            val = 0
            for row in range(x.shape[0]):
                if x[row,colx] == 0 and y[row,coly] == 1:
                    val += N
            if inv:
                if transpose:
                    j[2*coly+1,2*colx] = 1/val
                else:
                    j[2*colx,2*coly+1] = 1/val
            else:
                if transpose:
                    j[2*coly+1,2*colx] = val
                else:
                    j[2*colx,2*coly+1] = val

            val = 0
            for row in range(x.shape[0]):
                if x[row,colx] == 1 and y[row,coly] == 0:
                    val += N
            if inv:
                if transpose:
                    j[2*coly,2*colx+1] = 1/val
                else:
                    j[2*colx+1,2*coly] = 1/val
            else:
                if transpose:
                    j[2*coly,2*colx+1] = val
                else:
                    j[2*colx+1,2*coly] = val

            val = 0
            for row in range(x.shape[0]):
                if x[row,colx] == 1 and y[row,coly] == 1:
                    val += N
            if inv:
                if transpose:
                    j[2*coly+1,2*colx+1] = 1/val
                else:
                    j[2*colx+1,2*coly+1] = 1/val
            else:
                if transpose:
                    j[2*coly+1,2*colx+1] = val
                else:
                    j[2*colx+1,2*coly+1] = val

#Calculation of the joint distribution of a sets of binary variables, X, Y, and Z

#X is the mxn matrix of observations x classes, where Xij, codes the presence 
#of a stimulus of class j at observation i

#Y is the mxn matrix of observations x voxels, where Yij, codes the presence 
#of some binary feature for voxel j at observation i
#Y contains all of the voxels which may contain information (i.e. with some mask of interest)

#Z is the mxn matrix ov observations x voxels, where Zij coes the presence
#of some binary feature for voxel j at observation i
#Z contains all of the voxels which have already been selected as features

#The output is stored in the matrix j of the form
#( 4*Nclasses, 2*Nvoxels, NSelectedVoxels ) or ( 4*x.shape[1], 2*y.shape[1], z.shape[1] )

#P( X = 0, Y = 0, Z = 0 ) will be stored at 4*Nx,   2*Ny,   Nz
#P( X = 0, Y = 0, Z = 1 ) will be stored at 4*Nx+1, 2*Ny,   Nz
#P( X = 1, Y = 0, Z = 0 ) will be stored at 4*Nx+2, 2*Ny,   Nz
#P( X = 1, Y = 0, Z = 1 ) will be stored at 4*Nx+3, 2*Ny,   Nz
#P( X = 0, Y = 1, Z = 0 ) will be stored at 4*Nx,   2*Ny+1, Nz
#P( X = 0, Y = 1, Z = 1 ) will be stored at 4*Nx+1, 2*Ny+1, Nz
#P( X = 1, Y = 1, Z = 0 ) will be stored at 4*Nx+2, 2*Ny+1, Nz
#P( X = 1, Y = 1, Z = 1 ) will be stored at 4*Nx+3, 2*Ny+1, Nz

#The following function calculates the marginal using a single CPU
def joint3CPU( x, y, z, j ):
    
    N = (1/float(x.shape[0]))
    
    for colx in range(x.shape[1]):
        for coly in range(y.shape[1]):
            for colz in range(z.shape[1]):
                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 0 and y[row,coly] == 0 and z[row,colz] == 0:
                        val += N
                    j[ 4*colx, 2*coly, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 0 and y[row,coly] == 0 and z[row,colz] == 1:
                        val += N
                    j[ 4*colx+1, 2*coly, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 0 and y[row,coly] == 1 and z[row,colz] == 0:
                        val += N
                    j[ 4*colx, 2*coly+1, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 0 and y[row,coly] == 1 and z[row,colz] == 1:
                        val += N
                    j[ 4*colx+1, 2*coly+1, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 1 and y[row,coly] == 0 and z[row,colz] == 0:
                        val += N
                    j[ 4*colx+2, 2*coly, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 1 and y[row,coly] == 0 and z[row,colz] == 1:
                        val += N
                    j[ 4*colx+3, 2*coly, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 1 and y[row,coly] == 1 and z[row,colz] == 0:
                        val += N
                    j[ 4*colx+2, 2*coly+1, colz ] = val

                val = 0
                for row in range(x.shape[0]):
                    if x[row,colx] == 1 and y[row,coly] == 1 and z[row,colz] == 1:
                        val += N
                    j[ 4*colx+3, 2*coly+1, colz ] = val

#Calculation of the marginal distribution of a set of binary variables 
#X across a set observations (the rows) stored in m. 

#If the inverse is true, then reciprocal values of P(X=i) 
#for i element of {0,1} are returned

#If the tranpose is true, then the vector m must have shape (1, NumVar*2)
#Otherwise the vector m must have the shape (NumVar*2, 1)

#The probability that Var N = 0 will be stored at Nx*2
#The probability that Var N = 1 will be stored at Nx*2 + 1

#The following function calculates the marginal using a single CPU
def marginalCPU( x, m, inv, transpose ):
    
    N = float(x.shape[0])
    for colx in range(x.shape[1]):
        val = 0
        for row in range(x.shape[0]):
            val += x[row,colx]
        if inv:
            if transpose:
                m[0,2*colx] = N/(N-val)
                m[0,2*colx+1] = N/val
            else:
                m[2*colx,0] = N/(N-val)
                m[2*colx+1,0] = N/val
        else:
            if transpose:
                m[0,2*colx] = (N-val)/N
                m[0,2*colx+1] = val/N
            else:
                m[2*colx,0] = (N-val)/N
                m[2*colx+1,0] = val/N

#Calculation of MI of two sets binary variables X and Y on a sinlge cpu
#Xij is the presence or absence of stimulus j for instance i
#Yij is the presence or absence of a partiuclar binary feature at voxel j for instance i
#This function uses a single CPU
def MICPU( x, y ):
    
    import numpy as np
    
    #Define empty variables and number of instances
    Nx                       = x.shape[1]
    Ny                       = y.shape[1]
    mxrec                    = np.zeros((1   , 2*Nx),dtype=np.float32)
    myrec                    = np.zeros((2*Ny, 1   ),dtype=np.float32)
    jxy                      = np.zeros((2*Nx, 2*Ny),dtype=np.float32)
    mxrec_X_jxy_X_myrec      = np.zeros((Nx  , Ny  ),dtype=np.float32)
    
    #Define number of blocks and threads, and compute reciprocal of the marginals 
    marginalCPU( x, mxrec, 1, 1 )
    marginalCPU( y, myrec, 1, 0 )

    #Compute the joint distribution
    jointCPU( x, y, jxy, 0, 0 )

    #Compute ∑∑p(x,y)/(p(x)*p(y))
    computeSumMICPU( mxrec, jxy, myrec, mxrec_X_jxy_X_myrec )
    
    #Compute log2 transform and return
    return np.log2( mxrec_X_jxy_X_myrec )

#Calculation of CMI on a single CPU
def CMICPU( x, y, z ):
    
    import numpy as np
    
    #Define empty variables based on input shapes
    Nx                              = x.shape[1]
    Ny                              = y.shape[1]
    Nz                              = z.shape[1]

    mz                              = np.zeros((2*Nz, 1       ),dtype=np.float32)
    jxzrec                          = np.zeros((2*Nx, 2*Nz    ),dtype=np.float32)
    jyzrec                          = np.zeros((2*Ny, 2*Nz    ),dtype=np.float32)
    jxyz                            = np.zeros((4*Nx, 2*Ny, Nz),dtype=np.float32)
    jxzrec_X_jxyz_X_mz_X_jyzrec     = np.zeros((Nx  , Ny  , Nz),dtype=np.float32)

    #Compute the marginal distribution of z
    marginalCPU( z, mz, 0, 0 )
    
    #Compute the inverse joint distributions
    jointCPU( x, z, jxzrec, 1, 0 )
    jointCPU( y, z, jyzrec, 1, 0 )
    
    #Compute the triple joint distribution
    joint3CPU( x, y, z, jxyz )
    
    #Compute ∑∑∑p(x,y,z)*p(z)/(p(x,z)*p(y,z)) over all possible x y and z
    computeSumCMICPU(jxzrec, jxyz, jyzrec, mz, jxzrec_X_jxyz_X_mz_X_jyzrec )

    #Perform log2 transform on cpu and return CMI
    return np.log2( jxzrec_X_jxyz_X_mz_X_jyzrec )


#------------------Generalized functions for MCMI feature selection of binary variables in MRI images------------------

#Mutual information M(X;Y) for a set of stimuli instances in x and voxels y
def MI( x, y, gpu = False ):
    if gpu:
        return MIGPU( x, y )
    else:
        return MICPU( x, y )

#Conditional mutual information M(X;Y|Z) for a set of predictors/voxels y, and a set of already selected predictors z
def CMI( x, y, z, gpu = False ):
    if gpu:
        return CMIGPU( x, y, z  )
    else:
        return CMICPU( x, y, z )

#Minimum conditional mutual information M(X;Y|Z) for a set of predictors/voxels y, and a set of already selected predictors z
def MCMI( x, y, z, gpu = False):
    if gpu:
        return np.argmin( CMIPGU( x, y, z ), axis = 2 )
    else:
        return np.argmin( CMICPU( x, y, z ), axis = 2 ) 

#Select N_feat features per stimulus class which have the highest MI or CMI upon each iteration of the algorithm
#This will results in a maximum of Num_Stimulus_Classes*N_feat number of total features.
#However, if on a single iteration of the algorithm a particular feature has the highest MI/MCMI for two stimulus 
#or more stimulus classes, the total number of features will be less than Num_Stimulus_Classes*N_feat
def FeatSel( x, y, N_feat, gpu  = False ):
    
    #The first feature is selected to be the voxel with the maximum mutal information per word (x.shape[1])
    feats = np.argmax( MI( x, y, gpu ), axis = 1).reshape( 1, x.shape[0] )
    
    #Calculate the MCMI for X and Y given the selected features with the highest MI for each word
    #The z variable are all of the observations for (rows) for the voxel numbers 
    #which have the maximum MI or CMI for each word stored in feats
    mcmi  = MCMI( x, y, z=y[ :, feats ], gpu  )

    #Add/Concatenate those features which have the highest minimal conditional mutual information for each word 
    #To the existing feature set
    feats = np.concatenate( ( feats, np.argmax(mcmi, axis = 1 ).reshape( 1, x.shape[0] ) ) , axis = 1 ) 
        
    for i in range(N_feat - 2):
        
        #Concatenate minimum CMI from previous set of added predictors and then take the new minimum 
        #of the previous set plus the new set
        #This way mcmi always contains the minimum of MI(X;Y|Z) for all z
        mcmi  = np.argmin( np.concatenate( 
                                           (mcmi, MCMI( x, y, z=y[ :, feats[ 0, -1*x.shape[0] ] ], gpu  ) ), 
                                            axis = 2), 
                          axis = 2 )
        
        #Add/Concatenate those features which have the highest minimal conditional mutual information for each word 
        #To the existing feature set
        feats = np.concatenate( ( feats, np.argmax(mcmi, axis = 1 ).reshape( 1, x.shape[0] ) ) , axis = 1 ) 

    return feats

In [135]:
mi_gpu = MI( x, y, gpu = True)
mi_cpu = MI( x, y )
print np.array_equal(mi_gpu,mi_cpu)
print sum(sum(mi_gpu-mi_cpu))
print np.argmax(mi_gpu,axis=1) == np.argmax(mi_cpu,axis=1)
print zip(np.argmax(mi_gpu,axis=1),np.argmax(mi_cpu,axis=1))

False
0.00036084651947
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
[(14474, 14474), (31371, 31371), (2095, 2095), (27952, 27952), (29783, 29783), (206, 206), (8822, 8822), (685, 685), (27426, 27426), (21548, 21548), (1380, 1380), (8921, 8921), (32514, 32514), (206, 206), (9801, 9801), (11143, 11143), (24400, 24400), (27043, 27043), (8199, 8199), (20975, 20975), (15124, 15124), (372, 372), (15138, 15138), (8201, 8201), (31974, 31974), (32309, 32309), (21079, 21079), (26687, 26687), (3525, 3525), (20243, 20243), (11921, 11921), (25862, 25862), (29668, 29668), (27480, 27480), (4573, 4573), (16803, 16803), (23845, 23845), (9061, 9061), (8984, 8984), (10227, 10227)]


In [15]:
cmi_gpu = CMI( x, y, z, gpu = True)
cmi_cpu = CMI( x, y, z )
print np.array_equal(cmi_gpu,cmi_cpu)
print sum(sum(sum(cmi_gpu-cmi_cpu)))
print      np.argmax(np.argmin(cmi_gpu,axis=2),axis=1) ==  np.argmax(np.argmin(cmi_cpu,axis=2),axis=1)
print zip( np.argmax(np.argmin(cmi_gpu,axis=2),axis=1),    np.argmax(np.argmin(cmi_cpu,axis=2),axis=1) )

NameError: name 'cmi_cpu' is not defined